In [ ]:
import coi_maps
import fetch
import geopandas as gpd
from tqdm import tqdm

In [ ]:
# pull csv down from endpoints
ids_url = "https://kkttgpf0ja.execute-api.us-east-2.amazonaws.com/citygate/submissions/districtr-ids/michigan"
coi_url = "https://kkttgpf0ja.execute-api.us-east-2.amazonaws.com/citygate/submissions/csv/michigan?type=coi&length=100000"

coi_df = fetch.coi_submissions(ids_url, coi_url)

In [ ]:
coi_df.head()

In [ ]:
cois = coi_maps.assignment_to_shape(coi_df)

In [ ]:
cois.head()

In [ ]:
# Number of total communities submitted (>= number of submissions)
len(set(cois['id']))

In [ ]:
dissolved = cois.dissolve('id')

In [ ]:
dissolved = dissolved.drop('tile_id', axis = 1)
dissolved = dissolved.reset_index()

In [ ]:
# add the columns from the initial API coi_df
dissolved = dissolved.merge(coi_df, on='plan_id')
# clean up some column names
dissolved['id'] = dissolved['id_x']
dissolved['submission_id'] = dissolved['id_y']
dissolved = dissolved.drop(['id_x','id_y'], axis = 1)
# since shapefiles can't have datetime
dissolved['datetime'] = dissolved['datetime'].apply(str)

# move id (primary key) to front
primary_key = dissolved.pop('id')
dissolved.insert(0, 'id', primary_key)

In [ ]:
# unpack area text for each community
# each submission may contain multiple areas, so we want 
# to put the title and text for each area in the shapefile also
def get_area_text(p):
    row = dissolved[dissolved['id'] == p].iloc[0]
    community = int(p.split('-')[1])
    districtr = row['districtr_data']
    try:
        return districtr['plan']['parts'][community]['description']
    except KeyError:
        return "" # if description is empty

def get_area_title(p):
    row = dissolved[dissolved['id'] == p].iloc[0]
    community = int(p.split('-')[1])
    districtr = row['districtr_data']
    return districtr['plan']['parts'][community]['name']
    
dissolved['area_text'] = dissolved['id'].apply(get_area_text)
dissolved['area_title'] = dissolved['id'].apply(get_area_title)

In [ ]:
dissolved.head()

In [ ]:
# user will have to change outfile location
outfile = "../Michigan/mi_cois.shp"
dissolved.to_file(outfile)

In [ ]:
len(dissolved)

In [ ]:
# to create one geojson per community
geojson_path = '../Michigan/geojsons'
for idx, row in tqdm(dissolved.iterrows(), total=len(dissolved)):
    gpd.GeoSeries(row['geometry']).to_file(f"{geojson_path}/{row['id']}.geojson", driver="GeoJSON")